# Set-up

In [0]:
#@title Link to Google Drive
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass

!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

# Create directory called 'drive'
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
cd /content/drive/CSML Project

In [0]:
#@title install additional packages
!pip install openpyxl
!pip install imageio
!pip install shapely
!pip install contrastive

In [0]:
#@title import packages
import matplotlib.pyplot as plt
import pandas as pd
import os
import scipy.misc
import numpy as np
#from keras.applications import vgg16 #, inception_v3, resnet50, mobilenet
import time
import openpyxl
import seaborn as sns
#import zipfile
import imageio
from skimage.transform import resize
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, \
precision_recall_fscore_support, classification_report

from keras.backend import expand_dims
from keras.layers import Conv2D, Dense, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling2D, Flatten
from keras.layers import Input, Dense, concatenate
from keras.layers import BatchNormalization, Activation
from keras.models import Model
from keras import optimizers
from keras.models import load_model
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.utils import to_categorical

from IPython.display import clear_output
import zipfile

import statsmodels.api as sm

In [0]:
#@title import sample labels

wb = openpyxl.load_workbook('file_list.xlsx', read_only=True, data_only=True)
ws = wb['file_list']
labels = {}
for i in range(1, ws.max_row+1):
    fname = ws['A'+str(i+1)].value
    if fname is None:
        break
    labels[fname] = {'Grade': ws['F'+str(i+1)].value, 'Exclude': ws['G'+str(i+1)].value}
wb.close()

labels = pd.DataFrame(labels).transpose()

# XML data formatting

In [0]:
#@title import additional packages
import matplotlib.image as img
import xml.etree.ElementTree as ET

In [0]:
#@title define XML2DataFrame object
class XML2DataFrame:

    def __init__(self, xml_fname):
        with open(xml_fname, 'r') as myfile:
            xml_data=myfile.read()
        self.root = ET.XML(xml_data)

    def parse_root(self, root):
        return [self.parse_element(child) for child in iter(root) if child.tag[-7:] == 'Results']

    def parse_element(self, element, parsed=None):
        if parsed is None:
            parsed = dict()
        for key in element.keys():
            parsed[key] = element.attrib.get(key)
        if element.text:
            parsed[element.tag] = element.text
        for child in list(element):
            self.parse_element(child, parsed)
        return parsed

    def process_data(self):
        structure_data = self.parse_root(self.root)
        return pd.DataFrame(structure_data)

In [0]:
#@title format xml files one by one
xml_df_list = []
i = 0
n = 1
start_time = time.time()
for file in os.listdir('Nisch Pleomorphic Sarcomas'):
    if file.endswith(".param"):
        xml2df = XML2DataFrame('Nisch Pleomorphic Sarcomas\\'+file)
        xml_df = xml2df.process_data()
        xml_df = xml_df.assign(fname=file[:-6])
        print(n, ":", file[:-6], ":", xml_df.shape, ":", i, "-", i+xml_df.shape[0])
        i += xml_df.shape[0]
        n += 1
        col_names = [k.replace('{http://tempuri.org/CellParametersDS.xsd}','') for k in xml_df.columns]
        xml_df.columns = col_names
        xml_df_list.append(xml_df)
xml_all = pd.concat(xml_df_list,sort=False).reset_index(drop=True)
print(time.strftime("%H:%M:%S", time.gmtime(time.time() - start_time)))

# save

#xml_all.to_csv('xml_all.csv', sep=',')

In [0]:
#@title delete redundant features
xml_all = pd.read_csv('xml_all.csv', index_col='GUID')

del xml_all['Unnamed: 0']
del xml_all['Results']
del xml_all['STACKGUID']

for col in xml_all.columns:
    if len(xml_all[col].unique())==1:
        print(col)
        del xml_all[col]

# Normalization

In [0]:
#@title read in XML data
xml_all = pd.read_csv('xml_all.csv', index_col='GUID')

# delete entries from rubbish galleries
xml_all.drop(index=xml_all[xml_all['GALLERY']>3].index, inplace=True)

In [0]:
#@title define function to find the leftmost peak over a threshold
def find_leftmost_peak(hist, bin_width, threshold): 
    
    hist = hist[:(1000//bin_width)]
    if max(hist)<(threshold/bin_width):
        threshold /= 2
    hist_modes = (hist >= np.insert(hist[1:], -1, 1)) & (hist >= np.insert(hist[:-1], 0, 1)) & (hist >= (threshold/bin_width))
    index = list(np.arange(len(hist))[hist_modes])
    
    i = 0
    while i<len(index)-1 and len(index)>1:
        #if index[i+1]-index[i]<=int(index[i+1]*2/5):
        if np.sum(hist[index[i]:index[i+1]]<threshold/bin_width)==0: 
            if hist[index[i]]>=hist[index[i+1]]:
                index.pop(i+1)
            else:
                index.pop(i)
        else:
            i += 1
    #print(index)
    return index[0]

In [0]:
#@title identify the diploid peak (leftmost peak above the 10%)

bin_width = 25
bins = np.arange(0, 4000, bin_width)

IOD = xml_all[['fname', 'GALLERY', 'IOD']].copy()

for fname in IOD['fname'].unique():
    plt.clf()
    plt.subplots(figsize=(9, 5))
    # plot all cells
    hist, _ = np.histogram(np.clip(IOD.loc[IOD['fname']==fname]['IOD'], a_min=bins[0], a_max=bins[-1]), 
                           bins=bins, density=True)

    plt.bar(bins[1:], hist*bin_width, width=bin_width, alpha=0.7)

    # plot normal cells
    if IOD.loc[(IOD['fname']==fname) & (IOD['GALLERY']==2)].shape[0]>30:
        hist_normal, _ = np.histogram(np.clip(IOD.loc[(IOD['fname']==fname) & (IOD['GALLERY']==2)]['IOD'], a_min=bins[0], a_max=bins[-1]), 
                                      bins=bins, density=True)
        plt.bar(bins[1:], hist_normal*bin_width, width=bin_width, alpha=0.7)

    # identify diploid peak
    normal_mode = hist_normal.argmax()
    peak_pos = find_leftmost_peak(hist, bin_width, 0.1)
    plt.bar(bins[1:], np.asarray([hist[i] if i==peak_pos else 0 for i in range(len(hist))])*bin_width, 
            width=bin_width, color='red')

    plt.legend(['All cells', 'Normal cells (PWS)', 'identified dipoid peak'])
    plt.title(fname + ' Ave normal IOD: ' + str(np.average(IOD.loc[(IOD['fname']==fname) & (IOD['GALLERY']==2)]['IOD'])))

    print(IOD.loc[(IOD['fname']==fname) & (IOD['GALLERY']==2)].shape[0], ' normal cells')
    
    if labels.loc[fname, 'Exclude']=='Yes':
        pic_name = '(' + labels.loc[fname, 'Grade'] + ') ' + fname + ' (Excluded)'
    else:
        pic_name = '(' + labels.loc[fname, 'Grade'] + ') ' + fname
    plt.savefig('Visuals\\IOD Histogram\\Identify diploid\\' + pic_name + '.png')
    plt.show()
    #break

In [0]:
#@title collect the location of the identified diploid peak

bin_width = 25
bins = np.arange(0, 4000, bin_width)

IOD = xml_all[['fname', 'GALLERY', 'IOD']].copy()

peak_positions = {}

for fname in IOD['fname'].unique():
    # plot all cells
    hist, _ = np.histogram(np.clip(IOD.loc[IOD['fname']==fname]['IOD'], a_min=bins[0], a_max=bins[-1]), 
                           bins=bins, density=True)
    
    peak_pos = find_leftmost_peak(hist, bin_width, 0.025)
    peak_positions[fname] = {}
    peak_positions[fname]['peak_l'] = bins[peak_pos]
    peak_positions[fname]['peak_r'] = bins[1+peak_pos]

peak_dict = pd.DataFrame.from_dict(data=peak_positions, orient='index')

In [0]:
#@title get the AREA of the cells corresponding to the peak

area_dict = pd.merge(peak_dict, xml_all[['fname', 'IOD', 'AREA']], left_index=True, right_on='fname')
area_dict['peak_indicator'] = (area_dict['IOD']>area_dict['peak_l']) & (area_dict['IOD']<area_dict['peak_r'])
area_dict.drop(index=area_dict.loc[area_dict['peak_indicator']==False].index, inplace=True)

In [0]:
#@title find the sample normalizers for IOD and AREA and save
normalizer = {}

IOD = xml_all[['fname', 'GALLERY', 'IOD']].copy()
IOD = pd.merge(peak_dict, IOD, left_index=True, right_on='fname')
IOD['peak_indicator'] = (IOD['IOD']>IOD['peak_l']) & (IOD['IOD']<IOD['peak_r'])

for fname in peak_dict.index:
    normalizer[fname] = {}
    normalizer[fname]['IOD_normalizer'] = np.median(xml_all.loc[(IOD['fname']==fname) & (IOD['peak_indicator']==True)]['IOD'])
    normalizer[fname]['AREA_normalizer'] = np.median(xml_all.loc[(IOD['fname']==fname) & (IOD['peak_indicator']==True)]['AREA'])
normalizer = pd.DataFrame.from_dict(normalizer, orient='index')

normalizer.to_csv('normalizer.csv', sep=',')

# Create a subset of data

In [0]:
#@title Randomly choose a balanced subset of data

# read in all data
xml_all = pd.read_csv('xml_all.csv', index_col='GUID')
xml_all.drop(index=xml_all.loc[xml_all['GALLERY']>=4].index, inplace=True)
xml_all
if 'Grade' not in xml_all.columns:
    xml_all = pd.merge(xml_all, labels, left_on='fname', right_index=True)
xml_all = xml_all.drop(index=xml_all.loc[xml_all['Exclude']=='Yes'].index).drop(columns='Exclude')

# sample some nuclei in each tumour grade
temp = []
nums = {'High': 5000, 'Low': 5000, 'Intermediate': 4206}
for grade in ['High', 'Low', 'Intermediate']:
    sample_index = np.random.choice(xml_all.loc[xml_all['Grade']==grade].shape[0], 
                                    nums[grade], replace=False)
    temp.append(xml_all.loc[xml_all['Grade']==grade].iloc[sample_index])

# normal samples
normal_list = list(xml_all.loc[xml_all['Grade']=='Normal']['fname'].unique())
normal_dict = {}
for k in normal_list:
    if k in ['11 2946 4', '44   14 2896 6', '14 1380 C']:
        normal_dict[k] = 'NA'
    else:
        normal_dict[k] = k[8:]
normal_dict = pd.DataFrame.from_dict(normal_dict, orient='index')
normal_dict.columns=['Normal Type']

# lymph node sample
exclude = ['18 2301 Testis', '14 2565 Lymph node', '11 2946 4']
normal_df = pd.merge(xml_all.loc[(xml_all['Grade']=='Normal') & (xml_all['fname'].map(lambda x: x not in exclude))], 
                     normal_dict, how='left', left_on='fname', right_index=True)

N = 5000
sample_index = np.random.choice(normal_df.shape[0], N, replace=False)
temp.append(normal_df.iloc[sample_index])
    
temp = pd.concat(temp)

# PCA Visualization

In [0]:
#@title Run PCA and plot
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(df_data.drop(['fname', 'GALLERY', 'Grade', 'Normal Type'], axis=1).values)

pca = PCA(n_components=2)
pca_result = pca.fit_transform(x_scaled)

df_pca = df_data[['Grade', 'IOD']].copy()

df_pca['pca-one'] = pca_result[:,0]
df_pca['pca-two'] = pca_result[:,1]

# plot with no color

grades = ['High','Intermediate','Low','Normal']
sns.lmplot('pca-one', 'pca-two', data=df_pca,col='Grade', markers='x', 
           col_order=grades, fit_reg=False, scatter_kws={'alpha':0.2})
plt.suptitle('PCA Output')
plt.subplots_adjust(top=0.85)
plt.show()

# t-SNE Visualization

In [0]:
current_palette = sns.color_palette()

In [0]:
#@title normalize

def normalize(df):
    normalizer = pd.read_csv('normalizer.csv', index_col=0)
    df_copy = pd.merge(df, normalizer, left_on='fname', right_index=True)

    # normalize IOD and AREA by normalizers

    df_copy['IOD'] /= df_copy['IOD_normalizer']
    df_copy['AREA'] /= df_copy['AREA_normalizer']
    
    df_copy.reindex(df.index)
    
    df_copy['IOD'] = np.log2(df_copy['IOD'])
    df_copy['AREA'] = np.log2(df_copy['AREA'])
    
    # remove the normalizer columns
    del df_copy['IOD_normalizer']
    del df_copy['AREA_normalizer']
    return df_copy

df_data = normalize(temp)

In [0]:
#@title Scale, run t-SNE and visualize

# transform using min-max scaler

time_start = time.time()

min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(df_data.drop(['fname', 'GALLERY', 'Grade', 'Normal Type'], axis=1).values)

# run t-SNE

tsne = TSNE(n_components=2, verbose=1, n_iter=3000, init='pca', perplexity=30)
tsne_results = tsne.fit_transform(x_scaled)
print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

df_tsne = df_data.copy()
df_tsne['x-tsne'] = tsne_results[:,0]
df_tsne['y-tsne'] = tsne_results[:,1]

# draw plot (no color)

sns.set(font_scale=1.2, palette=current_palette)

grades = ['High','Intermediate','Low','Normal']
sns.lmplot('x-tsne', 'y-tsne', data=df_tsne, col='Grade', markers='x',
           col_order=grades, fit_reg=False, scatter_kws={'alpha':0.2})
plt.suptitle('t-SNE Output (perplexity=%d)' % 30)
plt.subplots_adjust(top=0.85)
plt.show()

In [0]:
#@title Visualize and color by normalized IOD

df = df_data[['Grade', 'IOD']].copy()

df['x-tsne'] = df_tsne['x-tsne']
df['y-tsne'] = df_tsne['y-tsne']

xmin = df['x-tsne'].min()-5
xmax = df['x-tsne'].max()+5
ymin = df['y-tsne'].min()-5
ymax = df['y-tsne'].max()+5

cmap = sns.cubehelix_palette(as_cmap=True)
f,ax=plt.subplots(figsize=(27.5,5.5))
grades = ['High','Intermediate','Low','Normal']
for i in range(4):
    plt.subplot(1,4,i+1)
    if i==0:
        points = plt.scatter(df.loc[df['Grade']==grades[i]]['x-tsne'], df.loc[df['Grade']==grades[i]]['y-tsne'], marker='x',
                            c=np.clip(df.loc[df['Grade']==grades[i]]['IOD'],a_min=-1,a_max=3), s=50, cmap=cmap, alpha=0.2)
    else:
        plt.scatter(df.loc[df['Grade']==grades[i]]['x-tsne'], df.loc[df['Grade']==grades[i]]['y-tsne'],  marker='x',
                    c=np.clip(df.loc[df['Grade']==grades[i]]['IOD'],a_min=-1,a_max=3), s=50, cmap=cmap, alpha=0.2)
    plt.xlim(xmin,xmax)
    plt.ylim(ymin,ymax)
    plt.title('Grade: '+grades[i])
    
cbar_ax = f.add_axes([0.92, 0.15, 0.01, 0.7])
f.colorbar(points, cax=cbar_ax)
plt.suptitle('t-SNE Output coloured by normalized IOD (perplexity=30)')
plt.subplots_adjust(top=0.85)
plt.show()

# Contrastive PCA Visualization

In [0]:
#@title set up
from contrastive import CPCA

min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(df_data.drop(['fname', 'GALLERY', 'Grade', 'Normal Type'], axis=1).values)

cpca = CPCA(standardize=False)
foreground_data = x_scaled[df_tsne['Grade']!='Normal']
background_data = x_scaled[df_tsne['Grade']=='Normal']

cpca.fit(foreground_data, background_data)

In [0]:
#@title Try different values of alpha
proj, alphas = cpca.transform(x_scaled, alpha_selection='all', n_alphas=10, max_log_alpha=1, return_alphas=True)
df = df_tsne.copy()
for coords, alpha in zip(proj, alphas):
    df['x'] = coords[:,0]
    df['y'] = coords[:,1]
    sns.lmplot('x', 'y', data=df, col='Grade', hue='region', fit_reg=False)
    plt.suptitle('%.3f' % alpha)
    plt.xlim(df['x'].min(),df['x'].max())
    plt.xlim(df['y'].min(),df['y'].max())
    plt.show()

In [0]:
#@title select one value and plot
proj = cpca.transform(x_scaled, alpha_selection='manual', alpha_value=2.424)

df = df_data[['Grade', 'IOD']].copy()

df['x-cpca'] = proj[:,0]
df['y-cpca'] = proj[:,1]

xmin = df['x-cpca'].min()
xmax = df['x-cpca'].max()
ymin = df['y-cpca'].min()
ymax = df['y-cpca'].max()

# plot with no color

df = df_data[['Grade', 'IOD']].copy()

df['x-cpca'] = proj[:,0]
df['y-cpca'] = proj[:,1]

sns.set(font_scale=1.2)
grades = ['High','Intermediate','Low','Normal']
sns.lmplot('x-cpca', 'y-cpca', data=df,col='Grade', markers='x', 
           col_order=grades, fit_reg=False, scatter_kws={'alpha':0.2})
plt.suptitle('cPCA Output (alpha=%.3f)' % 2.424)
plt.subplots_adjust(top=0.85)
plt.show()


# colour by IOD
cmap = sns.cubehelix_palette(as_cmap=True)
f,ax=plt.subplots(figsize=(27.5,5.5))
grades = ['High','Intermediate','Low','Normal']
for i in range(4):
    plt.subplot(1,4,i+1)
    if i==0:
        points = plt.scatter(df.loc[df['Grade']==grades[i]]['x-cpca'], df.loc[df['Grade']==grades[i]]['y-cpca'], marker='x',
                            c=np.clip(df.loc[df['Grade']==grades[i]]['IOD'],a_min=-1,a_max=3), s=50, cmap=cmap, alpha=0.2)
    else:
        plt.scatter(df.loc[df['Grade']==grades[i]]['x-cpca'], df.loc[df['Grade']==grades[i]]['y-cpca'], marker='x',
                    c=np.clip(df.loc[df['Grade']==grades[i]]['IOD'],a_min=-1,a_max=3), s=50, cmap=cmap, alpha=0.2)
    plt.xlim(xmin,xmax)
    plt.ylim(ymin,ymax)
    plt.title('Grade: '+grades[i])
    
cbar_ax = f.add_axes([0.92, 0.15, 0.01, 0.7])
f.colorbar(points, cax=cbar_ax)
plt.suptitle('cPCA Output coloured by normalized IOD (alpha=2.424)')
plt.subplots_adjust(top=0.85)
plt.show()

# Label abnormal clusters

In [0]:
#@title specify vertices of the abnormal regions

coords1 = [(-100,100),(-66,40),(-3,60), (0,120), (120,160), (-100,160), (-80,120)]

coords3 = [(25,50),(35,47),(46,53),(55,83),(42,81),(33,73),(25,57)]

p = Point(5, -5)
x = p.buffer(15)
s = x.simplify(0.02, preserve_topology=False)

In [0]:
#@title label each point

# region 1
polygon1 = Polygon(coords1)
# region 3
polygon3 = Polygon(coords3)

df_tsne['region'] = np.zeros(df_tsne.shape[0], dtype=np.int32)
for guid in df_tsne.index:
    point = Point(df_tsne.loc[guid,'x-tsne'], df_tsne.loc[guid,'y-tsne'])
    if polygon1.contains(point):
        df_tsne.loc[guid,'region'] = 1
    elif s.contains(point):
        df_tsne.loc[guid,'region'] = 2
    elif polygon3.contains(point):
        df_tsne.loc[guid,'region'] = 3

In [0]:
#@title Visualize labels

grades = ['High','Intermediate','Low','Normal']
lm = sns.lmplot('x-tsne', 'y-tsne', data=df_tsne, col='Grade', 
           markers='x', hue='region',
           col_order=grades, fit_reg=False, scatter_kws={'alpha':0.3, 's':20}, size=4, aspect=0.9)
plt.suptitle('Target t-SNE Output (test data)')
plt.subplots_adjust(top=0.8)
for lh in lm._legend.legendHandles:
    lh.set_alpha(1)
    lh._sizes = [50]
plt.show()

# Train fully connected multi-task neural network
Learn the embedding function and classify nuclei into clusters at the same time

In [0]:
#@title Import and normalize data

# read saved data
temp = pd.read_csv('temporary files/data_final.csv', 
                   index_col='GUID', dtype={'Normal Type':str})
df_tsne = pd.read_csv('temporary files/outputs_tsne_final.csv', 
                      index_col='GUID', dtype={'Normal Type':str})

normalizer = pd.read_csv('normalizer.csv', index_col=0)

temp = pd.merge(temp, normalizer, left_on='fname', right_index=True)

# normalize IOD and AREA by normalizers

temp['IOD'] /= temp['IOD_normalizer']
temp['AREA'] /= temp['AREA_normalizer']

temp = temp.reindex(df_tsne.index)

# remove the normalizer columns
del temp['IOD_normalizer']
del temp['AREA_normalizer']
del normalizer

temp['IOD'] = np.log2(temp['IOD'])
temp['AREA'] = np.log2(temp['AREA'])
print(temp['IOD'].median(), temp['AREA'].median())

In [0]:
#@title scale inputs

df_data = df_tsne.drop(columns=['fname','GALLERY','Grade','PWS Classification','IODbin','region','Normal Type'])

min_max_scaler = preprocessing.MinMaxScaler()
for col in df_data.columns:
    if col not in ['x-tsne', 'y-tsne']:
        df_data[col] = min_max_scaler.fit_transform(df_data[col].values.reshape(-1, 1))

In [0]:
#@title For each grade, randomly sample 2/3 of the indices for training set, 1/6 for validation and remaining 1/6 for test
grade = 'High'
train_index = np.random.choice(temp.reset_index().loc[(temp['Grade']==grade).values].index.values, 
                               size=int(2/3*temp.loc[temp['Grade']==grade].shape[0]),  
                               replace=False)
for grade in ['Intermediate', 'Low', 'Normal']:
    train_index = np.concatenate([train_index, 
                                  np.random.choice(temp.reset_index().loc[(temp['Grade']==grade).values].index.values, 
                                                   size=int(2/3*temp.loc[(temp['Grade']==grade).values].shape[0]),  
                                                   replace=False)])

val_index = None
test_index = None
for grade in ['High', 'Intermediate', 'Low', 'Normal']:
    df = temp.reset_index().loc[(temp['Grade']==grade).values]
    remaining = list(set(df.index.values).difference(set(train_index)))
    sample_val = np.random.choice(remaining, int(0.5*len(remaining)), replace=False)
    sample_test = np.array(list(set(remaining).difference(set(sample_val))), dtype=np.int32)
    if val_index is None:
        val_index = sample_val
        test_index = sample_test
    else:
        val_index = np.concatenate([val_index, sample_val])
        test_index = np.concatenate([test_index, sample_test])
                     
#test_index = np.array(list(set(np.arange(temp.shape[0])).difference(set(train_index))), dtype=np.int32)

In [0]:
#@title save indices
np.save('temporary files/train_index.npy', train_index)
np.save('temporary files/val_index.npy', val_index)
np.save('temporary files/test_index.npy', test_index)

In [0]:
#@title retrieve indices
train_index = np.load('temporary files/train_index.npy')
val_index = np.load('temporary files/val_index.npy')
test_index = np.load('temporary files/test_index.npy')

In [0]:
#@title Define DataSet object

class DataSet(object):
    def __init__(self,
                 inputs,
                 labels,
                 aux_outputs=None,
                 aux_inputs=None,
                 balance_label=None):

        assert len(inputs) == len(labels), (
            'inputs len: %s labels.shape: %s' % (len(inputs), len(labels)))
        self._num_examples = inputs.shape[0]
        self._inputs = inputs
        self._labels = labels
        #self._epochs_completed = 0
        #self._index_in_epoch = 0
        self._index = np.arange(self._num_examples)
        self._aux_outputs = aux_outputs
        self._aux_inputs=aux_inputs
        self._b = balance_label

In [0]:
#@title use the indices to prepare data sets
index = train_index
train_data = DataSet(inputs=df_data.drop(columns=['x-tsne','y-tsne']).values[index],
                     labels=df_data[['x-tsne', 'y-tsne']].values[index],
                     aux_outputs=df_tsne['region'].values[index])
index = val_index
val_data = DataSet(inputs=df_data.drop(columns=['x-tsne','y-tsne']).values[index],
                     labels=df_data[['x-tsne', 'y-tsne']].values[index],
                     aux_outputs=df_tsne['region'].values[index])
index = test_index
test_data = DataSet(inputs=df_data.drop(columns=['x-tsne','y-tsne']).values[index],
                     labels=df_data[['x-tsne', 'y-tsne']].values[index],
                     aux_outputs=df_tsne['region'].values[index])

In [0]:
#@title Define architecture
layers = [61, 128, 256, 512, 1024, 1024, 512]
y_trail = [256, 256, 128, 64]
aux_trail = [256, 128]

In [0]:
#@title Define model with Keras

x_ = Input(shape=(layers[0],))

x = x_
i = 1
for n in layers[1:]:
    x = Dense(units=n, activation='relu', name='fc'+str(i)+'_'+str(n))(x)
    #if i<len(layers)-2:
    #    x = BatchNormalization()(x)
    #x = Activation('relu')(x)
    i += 1

n = aux_trail[0]
x_aux = Dense(units=n, activation='relu', name='aux_fc'+str(i)+'_'+str(n))(x)
if len(aux_trail)>1:
    for n in aux_trail[1:]:
        x_aux = Dense(units=n, activation='relu', name='aux_fc'+str(i)+'_'+str(n))(x_aux)
        i += 1
aux_output = Dense(units=4, activation='softmax', name='aux_output')(x_aux)

for n in y_trail:
    x = Dense(units=n, activation='relu', name='fc'+str(i)+'_'+str(n))(x)
    #if i<len(layers)-2:
    #    x = BatchNormalization()(x)
    #x = Activation('relu')(x)
    i += 1
y = Dense(units=2, name='fc_final')(x)

model = Model(inputs=x_, outputs=[y, aux_output])

In [0]:
#@title Define my f1 metric
import keras.backend as K
def myF1(y_true, y_pred):
    #val_targ = K.argmax(y_true) 
    val_predict = K.argmax(y_pred, axis=1)
    
    f1 = 0
    real_positives = K.sum(y_true, axis=0)
    for c in range(4):
        y_pred_c = K.cast(K.equal(val_predict, c), 'float32')
        
        true_positives = K.sum(y_true[:,c] * y_pred_c)
        prec = true_positives / (real_positives[c] + K.epsilon())
        
        predicted_positives = K.sum(y_pred_c)
        rec = true_positives / (predicted_positives + K.epsilon())
        
        f1 += 2 * (prec*rec) /(prec + rec + K.epsilon())
    return f1/4

In [0]:
model_name_prefix = 'NN_model_v0'
def reset_log():
    return {'train':[], 'val':[]}

In [0]:
#@title define metrics callback
from keras.callbacks import Callback

class Metrics(Callback):
    def on_train_begin(self, logs={}):
        self._f1score = 0.9
        if len(self.model.inputs)>1:
            self._aux_in = True
        else:
            self._aux_in = False
        
    def on_epoch_end(self, epoch, logs={}):
        if self._aux_in:
            val_predict = np.argmax(self.model.predict(self.validation_data[:2])[1], axis=1)
            n = 3
        else:
            val_predict = np.argmax(self.model.predict(self.validation_data[0])[1], axis=1)
            train_predict = np.argmax(self.model.predict(train_data._inputs)[1], axis=1)
            _train_f1 = f1_score(train_data._aux_outputs, 
                                 train_predict, average='macro')
            F1_log['train'].append(_train_f1)
            n = 2
        
        val_targ = np.argmax(self.validation_data[n],axis=1)
        _val_f1 = f1_score(val_targ, val_predict, average='macro')
        
        F1_log['val'].append(_val_f1)
                
        print(' — val_f1: %f' % _val_f1)
        if _val_f1>self._f1score:
            file = 'Keras models/NonCNN/'+model_name_prefix+'_f%.3f_%.3f.h5' % (_val_f1, logs['val_fc_final_loss'])
            print('saving model at '+file)
            self.model.save(file)
            self._f1score = _val_f1
        return

In [0]:
#@title Configure callbacks

reduce_lr = ReduceLROnPlateau(monitor='val_loss', mode='min', 
                              factor=0.5, patience=3, 
                              verbose=1, min_lr=1e-7)
metrics = Metrics()

In [0]:
#@title Compile model
F1_log = reset_log()
model.compile(optimizer='adam', #'RMSProp','adam'
              loss=['mean_absolute_error', 'categorical_crossentropy'], 
              loss_weights=[0.001, 1.],
              metrics={'aux_output': myF1}) #optimizers.Adam(lr=1e-3)

In [0]:
#@title Train

hist = model.fit(train_data._inputs, 
              [train_data._labels, to_categorical(train_data._aux_outputs, num_classes=4)], 
              epochs=500, batch_size=128, shuffle=True, 
              validation_data=(val_data._inputs, 
                               [val_data._labels, to_categorical(val_data._aux_outputs, num_classes=4)]),
              #sample_weight= [sample_weights] * 2,
              callbacks=[reduce_lr, metrics]) # , checkpointer ,initial_epoch=100

In [0]:
#@title plot learning curve for f1 score

plt.rc('xtick', labelsize=12)
plt.plot(F1_log['train'], label='on training set')
plt.rc('ytick', labelsize=12)
plt.plot(F1_log['val'], label='on validation set')
plt.xlabel('epoch',fontsize=15)
plt.ylabel('macro F1 score',fontsize=15)
plt.legend()
plt.ylim(min(0.6, min(F1_log['train'])-0.02, min(F1_log['val'])-0.02),1.02)
if model.loss_weights[0]==0:
    plt.title('Training log of macro F1 score (single-task)',fontsize=15)
else:
    plt.title('Training log of macro F1 score (multi-task)',fontsize=15)
plt.show()

In [0]:
#@title plot learning curve for mean absolute loss

plt.rc('xtick', labelsize=12)
plt.plot(hist.history['fc_final_loss'], label='on training set')
plt.rc('ytick', labelsize=12)
plt.plot(hist.history['val_fc_final_loss'], label='on validation set')
plt.xlabel('epoch',fontsize=15)
plt.ylabel('mean absolute loss',fontsize=15)
plt.ylim(2,20)
plt.legend()
plt.title('Training log of mean absolute loss',fontsize=15)
#plt.gca().set_ylim(3, 4)
plt.show()

In [0]:
#@title plot learning curve for categorical crossentropy loss

plt.rc('xtick', labelsize=12)
plt.semilogy(hist.history['loss'], label='on training set')
plt.rc('ytick', labelsize=12)
plt.semilogy(hist.history['val_loss'], label='on validation set')
plt.xlabel('epoch',fontsize=15)
plt.ylabel('cross entropy loss',fontsize=15)
#plt.rcParams.update({'font.size': 20})
plt.gca().set_ylim(0.001,1)
plt.legend()
if model.loss_weights[0]==0:
    plt.title('Training log of categorical cross entropy loss (single-task)',fontsize=15)
else:
    plt.title('Training log of categorical cross entropy loss (multi-task)',fontsize=15)
plt.show()

# Model evaluation

In [0]:
#@title Load model
model_name = 'NN_model_v35_f0.947_5.150'
model = load_model('Keras models/NonCNN/'+model_name, custom_objects={'myF1': myF1})

In [0]:
#@title evaluate on test data

outputs = model.predict(test_data._inputs, batch_size=128, verbose=1) #[:val_data._b.shape[0]]

df = df_tsne.iloc[test_index].copy()

df['x-model'] = outputs[0][:, 0]
df['y-model'] = outputs[0][:, 1]
df['region-model'] = np.argmax(outputs[1], axis=1)

print(np.mean(df['region-model']==df['region']))
print(f1_score(df['region'], df['region-model'], average='macro'))
print(classification_report(df['region'], df['region-model'], labels=[0,1,2,3],
                           target_names=['region'+str(i) for i in range(4)], digits = 3))
pd.crosstab(df['region-model'], df['region'])

In [0]:
model.evaluate(x=test_data._inputs, y=[test_data._labels, to_categorical(test_data._aux_outputs, num_classes=4)])

# Visualize model output

In [0]:
#@title plot the original t-SNE

sns.lmplot( x='x-tsne', y='y-tsne', data=df_tsne.iloc[test_index], fit_reg=False, 
            hue='region', col='Grade', col_order=['High','Intermediate','Low','Normal'],
            markers='x', scatter_kws={'alpha':0.4})

In [0]:
#@title plot the output (test data)

outputs = model.predict(test_data._inputs, batch_size=128, verbose=1)

df = df_tsne.iloc[test_index].copy()

df['x-model'] = outputs[0][:, 0]
df['y-model'] = outputs[0][:, 1]
df['region-model'] = np.argmax(outputs[1], axis=1)

sns.lmplot( x='x-model', y='y-model', data=df, fit_reg=False, 
            hue='region-model', col='Grade', col_order=['High','Intermediate','Low','Normal'],
            markers='x', scatter_kws={'alpha':0.4}) 
plt.show()

sns.lmplot( x='x-model', y='y-model', data=df, fit_reg=False, 
            hue='region', col='Grade', col_order=['High','Intermediate','Low','Normal'],
            markers='x', scatter_kws={'alpha':0.4}) 
plt.show()

# Get model outputs for all data

In [0]:
#@title read in all XML data
xml_all = pd.read_csv('xml_all.csv', index_col='GUID')

xml_all.drop(index=xml_all.loc[xml_all['GALLERY']>=4].index, inplace=True)

xml_all = pd.merge(labels, xml_all, left_index=True, right_on='fname')
xml_all = xml_all.drop(index=xml_all.loc[xml_all['Exclude']=='Yes'].index).drop(columns='Exclude')

xml_all = xml_all[df_tsne.drop(columns=['IODbin', 'Normal Type', 'PWS Classification', 'region', 'x-tsne', 'y-tsne']).columns.tolist()]

In [0]:
#@title normalize

normalizer = pd.read_csv('normalizer.csv', index_col=0)

modelOutputs = xml_all.copy()
modelOutputs = pd.merge(modelOutputs, normalizer, left_on='fname', right_index=True)

# normalize IOD and AREA by normalizers

modelOutputs['IOD'] /= modelOutputs['IOD_normalizer']
modelOutputs['AREA'] /= modelOutputs['AREA_normalizer']

modelOutputs = modelOutputs.reindex(xml_all.index)

# remove the normalizer columns
del modelOutputs['IOD_normalizer']
del modelOutputs['AREA_normalizer']
del normalizer

modelOutputs['IOD'] = np.log2(modelOutputs['IOD'])
modelOutputs['AREA'] = np.log2(modelOutputs['AREA'])
print(modelOutputs['IOD'].median(), modelOutputs['AREA'].median())

min_max_scaler = preprocessing.MinMaxScaler()
min_max_scaler.fit(temp.drop(['fname', 'GALLERY', 'Grade', 'Normal Type'], axis=1).values)

In [0]:
using_aux_output = True

In [0]:
#@title save model outputs for each sample

modelOutputs['x-model'] = np.zeros(modelOutputs.shape[0])
modelOutputs['y-model'] = np.zeros(modelOutputs.shape[0])
modelOutputs['region-model'] = np.zeros(modelOutputs.shape[0])
for a in range(4):
    modelOutputs['prop'+str(a)] = np.zeros(modelOutputs.shape[0])

batch_size = 128

# feed into model to obtain outputs
outputs = model.predict(min_max_scaler.transform(modelOutputs[xml_all.drop(columns=['fname','GALLERY', 'Grade']).columns.tolist()].values), 
                        batch_size=batch_size, verbose=1)

if using_aux_output:
    modelOutputs['x-model'] = outputs[0][:, 0]
    modelOutputs['y-model'] = outputs[0][:, 1]
    for a in range(4):
        modelOutputs['prop'+str(a)] = outputs[1][:,a]
    modelOutputs['region-model'] = np.argmax(outputs[1],axis=1)
else:
    modelOutputs['x-model'] = outputs[:, 0]
    modelOutputs['y-model'] = outputs[:, 1]
    
# save model outputs
modelOutputs[['x-model','y-model','region-model', 
              'fname', 'GALLERY', 'Grade'] + ['prop'+str(i) for i in range(4)]].to_csv('modelOutputs.csv', sep=',')

# Generate sample profiles

In [0]:
#@title get model outputs
modelOutputs = pd.read_csv('modelOutputs.csv', index_col='GUID')

print(modelOutputs['x-model'].min(), modelOutputs['x-model'].max())
print(modelOutputs['y-model'].min(), modelOutputs['y-model'].max())

In [0]:
xy = [(-180, 155), (-130, 140)]

In [0]:
#@title generate sample profiles

cwd = os.getcwd()
on_cloud = cwd.startswith(r'/home/liyunlu3')

for fname in xml_all['fname'].unique():
    
    # feed into CNN to obtain outputs
    df = modelOutputs.loc[modelOutputs['fname']==fname][['Grade', 'region-model', 'x-model', 'y-model']]
    
    sns.lmplot( x='x-model', y='y-model', data=df, fit_reg=False, hue='region-model',
                markers='x', scatter_kws={'alpha':0.2})
    title = '('+labels.loc[fname,'Grade']+' Grade) '+fname
    plt.xlim(xy[0][0], xy[0][1])
    plt.ylim(xy[1][0], xy[1][1])
    plt.title(title)
    plt.subplots_adjust(top=0.9)
    #plt.savefig('Visuals/sample profiles/'+title+'.png')
    plt.show()

# Integrating with clinical and genetic data

In [0]:
#@title Import data

# get neural network outputs for all nuclei
modelOutputs = pd.read_csv('modelOutputs.csv', index_col='GUID')

# get labels for high grade included samples only
wb = openpyxl.load_workbook('file_list.xlsx', read_only=True, data_only=True)
ws = wb['file_list']
labels = {}
for i in range(1, ws.max_row+1):
    fname = ws['A'+str(i+1)].value
    if fname is None:
        break
    labels[fname] = {'Grade': ws['F'+str(i+1)].value, 
                     'Exclude': ws['G'+str(i+1)].value,
                     'Pathology.No':ws['C'+str(i+1)].value[:7].replace(' ','--')}
wb.close()
del wb
del ws

labels = pd.DataFrame(labels).transpose()
labels.drop(index=labels.loc[(labels['Exclude']=='Yes')|(labels['Grade']!='High')].index, inplace=True)
labels.drop(columns=['Exclude', 'Grade'], inplace=True) 
# all matched samples must be high grade, so drop other grades


# get survival variables data

survival = pd.read_csv('survivalVariables.csv', index_col='Pathology.No')
survival.shape

In [0]:
#@title aggregate into sample level features
sample_features = pd.crosstab(modelOutputs['fname'], modelOutputs['region-model']).apply(lambda r: r/r.sum(), axis=1)
sample_features.rename(columns={0:'region0', 1:'region1', 2:'region2', 3:'region3'}, inplace=True)
print(sample_features.shape)

#sample_features = sample_features.merge(modelOutputs[['fname','Grade']].groupby('fname').agg('first'), left_index=True, right_index=True)
sample_features['region-all'] = sample_features['region1']+sample_features['region2']+sample_features['region3']
#plt.hist(sample_features['region3'], 30)
#plt.show()

In [0]:
#@title aggregate into tumour level features
tumour_features = pd.crosstab(modelOutputs['Pathology.No'], modelOutputs['region-model']).apply(lambda r: r/r.sum(), axis=1)
tumour_features.rename(columns={0:'region0', 1:'region1', 2:'region2', 3:'region3'}, inplace=True)
print(tumour_features.shape)

tumour_features['region-all'] = tumour_features['region1']+tumour_features['region2']+tumour_features['region3']
#plt.hist(tumour_features['region1'], 30)
#plt.show()

In [0]:
#@title save features
sample_features.to_csv('region_features.csv', sep=',')
tumour_features.to_csv('region_features_by_tumour.csv', sep=',')

In [0]:
#@title get Relative proportions

for col in ['region'+str(i) for i in range(1,4)]:
    combined[col] /= combined['region-all']

In [0]:
#@title plot scatter plot

region = 'region2'
feature = 'overallSurvivalTime'
cat = 'CDKN2A'

df = combined[[region, feature, cat, 'dead', 'Nature']].dropna(how='any').copy()
df[region] = np.sqrt(combined[region])
df[feature] = np.sqrt(combined[feature])
print(df[region].corr(df[feature]))
sns.lmplot(region, feature, data=df, fit_reg=False, hue='Nature')
plt.show()

for c in df[cat].unique():
    print(cat + ' ' + str(c) + ':', df[region].loc[df[cat]==c].corr(df[feature].loc[df[cat]==c]))
sns.lmplot(region, feature, data=df, col=cat, fit_reg=False, hue='Nature')
plt.show()

# Image Data Formatting

In [0]:
#@title Define function to split image
def split(directory):
    # read image
    image = img.imread(directory)
    
    # identify a mask based on RGB (all=1):
    mask0 = (image[:,:,0] < 1)
    
    # identify another mask based on aligned bounding box:
    # scan vertically
    mask1 = np.zeros(image.shape[:2], dtype=np.int64)
    row_mask = image[:,:,0].mean(axis=1)<1
    row_index, N_rows = ndimage.label(row_mask)
    
    # check vertical splits
    relabel = False
    for i in range(2,N_rows+1):
        n_start = np.arange(image.shape[0])[row_index==(i-1)].max()+1
        n_end = np.arange(image.shape[0])[row_index==i].min()
        if n_end - n_start<5:
            row_mask[n_start:n_end] = 1
            # print(n_start,n_end)
            relabel = True
    if relabel:
        row_index, N_rows = ndimage.label(row_mask)
    
    #scan horizontally
    n = 0
    for i in range(1,N_rows+1):
        row_height = np.sum(row_index==i)
        col_mask = (image[:,:,0][row_index==i].mean(axis=0) < 1) + 0
        
        # check horizontal splits
        col_index, N_cols = ndimage.label(col_mask)
        for j in range(2,N_cols+1):
            n_start = np.arange(image.shape[1])[col_index==(j-1)].max()+1
            n_end = np.arange(image.shape[1])[col_index==j].min()
            if n_end - n_start<5:
                col_mask[n_start:n_end] = 1
        mask1[row_index==i] = col_mask.reshape(1,-1)
    
    # use the second mask to identify the correct label for each patch
    # then use the first mask to refine the labelled patches
    label, _ = ndimage.label(mask1)
    label = np.multiply(label, mask0)
    
    # extract the slices from the labels
    slices = ndimage.find_objects(label)
    return [image[i_slice] for i_slice in slices]

In [0]:
#@title get the indices information

wb = openpyxl.load_workbook(main_path + 'Image data logistics.xlsx', read_only=True, data_only=True)
ws = wb.get_sheet_by_name('main')
indices = {}
for i in range(1, ws.max_row):
    fname = ws['A'+str(i+1)].value
    if ws['G'+str(i+1)].value is None:
        break
    indices[fname] = {}
    n = ws['F'+str(i+1)].value
    if n>1:
        indices[fname]['b'] = ws['I'+str(i+1)].value
        if n>2:
            indices[fname]['c'] = ws['K'+str(i+1)].value
            if n>3:
                indices[fname]['d'] = ws['M'+str(i+1)].value
wb.close()

In [0]:
#@title format and save Gallery 1 images folder by folder
suffices = ['a','b','c','d']
for folder in os.listdir(main_path + 'Raw data\\image data'):
    
    for file in os.listdir('Raw data\\image data\\' + folder):
        if file.endswith('.info'):
            fname = file[:-5]
            break
    if fname not in list(indices.keys()):
        print('file', fname, 'passed.')
        continue
    if os.path.exists(main_path + 'images\\' + fname):
        print(fname, 'already created.')
    else:
        images = []
        for suffix in suffices:
            path = main_path + "Raw data\\image data\\" + folder + "\\" + fname + suffix + ".png"
            if os.path.isfile(path): 
                images.append(split(path))
        if len(images) == 0:
            print("Folder " + folder + " does not contain image file, or their names are incorrect")
        else:
            # save images
            directory = main_path + 'images\\' + fname
            if not os.path.exists(directory):
                os.makedirs(directory)
            n = 0
            for i_sliced_im in images[0]:
                n += 1
                scipy.misc.imsave( directory + '\\' + fname + '_' + str(n) + '.png', i_sliced_im)
            if len(images)>1:
                
                for i in range(1,len(images)):
                    print(fname, suffices[i-1], len(images[0]), ' vs ', suffices[i], indices[fname][suffices[i]])
                    plt.subplot(121)
                    plt.imshow(images[i-1][-1])
                    plt.subplot(122)
                    plt.imshow(images[i][indices[fname][suffices[i]]-2])
                    plt.show()
                    for i_sliced_im in images[i][(indices[fname][suffices[i]]-1):]:
                        n += 1
                        scipy.misc.imsave( directory + '\\' + fname + '_' + str(n) + '.png', i_sliced_im)

In [0]:
#@title format and save Gallery 2-4 images
# get the counts information

wb = openpyxl.load_workbook(main_path + 'Image data logistics.xlsx', read_only=True, data_only=True)
ws = wb.get_sheet_by_name('main')
indices = {}
for i in range(1, ws.max_row):
    fname = ws['A'+str(i+1)].value
    indices[fname] = ws['B'+str(i+1)].value
wb.close()

# save images
stats = {}
for folder in os.listdir(main_path + 'Raw data\\image data'):
    for file in os.listdir('Raw data\\image data\\' + folder):
        if file.endswith('.info'):
            fname = file[:-5]
            break
    
    directory = main_path + 'images\\' + fname
    if not os.path.exists(directory):
        print(fname, ' excluded.')
    else:
        stats[fname] = {}
        n = len([name for name in os.listdir(directory)])
        stats[fname][1] = n
        for suffix in [2,3,4]:
            path = main_path + "Raw data\\image data\\" + folder + "\\" + fname + '_G' + str(suffix) + ".png"
            if os.path.isfile(path): 
                images = split(path)
                stats[fname][suffix] = len(images)
                # save images
                for i_sliced_im in images:
                    n += 1
                    scipy.misc.imsave( directory + '\\' + fname + '_' + str(n) + '.png', i_sliced_im)
            else:
                stats[fname][suffix] = None

In [0]:
#@title match xml data with image names

xml_all = pd.read_csv('xml_all.csv', index_col='GUID')
xml_all.drop(index=xml_all.loc[xml_all['GALLERY']>=4].index, inplace=True)

#get labels
wb = openpyxl.load_workbook('file_list.xlsx', read_only=True, data_only=True)
ws = wb['file_list']
labels = {}
for i in range(1, ws.max_row+1):
    fname = ws['A'+str(i+1)].value
    if fname is None:
        break
    labels[fname] = {'Grade': ws['F'+str(i+1)].value, 'Exclude': ws['G'+str(i+1)].value}
wb.close()

labels = pd.DataFrame(labels).transpose()
#labels = labels.drop(index=labels[labels['Exclude']=='Yes'].index).drop(columns='Exclude')

xml_all = pd.merge(labels, xml_all, left_index=True, right_on='fname')
xml_all = xml_all.drop(index=xml_all.loc[xml_all['Exclude']=='Yes'].index).drop(columns='Exclude')

dfs = []

for fname in np.unique(xml_all['fname']):
    temp_df = xml_all[['fname', 'GALLERY', 'IOD']].loc[xml_all['fname']==fname].copy()
    # the images are arranged in gallery and IOD ascneding order
    temp_df.sort_values(['GALLERY', 'IOD'], ascending=[True, True], inplace=True)
    temp_df.reset_index(inplace=True)
    temp_df.index += 1
    dfs.append(temp_df)
dfs = pd.concat(dfs)

dfs['image'] = dfs['fname'] + '_' + dfs.index.map(str)

index_image_table = dfs[['GUID', 'image', 'GALLERY']].copy()
index_image_table.set_index('GUID', inplace=True)

# save
#index_image_table.to_csv('index_to_image_lookup.csv')

# Extension: convolutional neural network
Directly work with raw images

In [0]:
#@title upload images data

zip_ref = zipfile.ZipFile('images.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [0]:
#@title Define function to standardize images

def standardize_image(im, max_length):
    h, w = im.shape
    
    
    if h>max_length or w>max_length:
        # downsize to fit
        
        # add padding to the shorter side
        if h>w:
            w_pad_before = int(np.floor((h - w)/2))
            w_pad_after = int(h - w - w_pad_before)
            im = np.pad(im, ((0, 0), (w_pad_before, w_pad_after)), mode='constant', constant_values=255)
        elif h<w:
            h_pad_before = int(np.floor((w - h)/2))
            h_pad_after = int(w - h - h_pad_before)
            im = np.pad(im, ((h_pad_before, h_pad_after), (0, 0)), mode='constant', constant_values=255)
        
        # scale down        
        im = resize(image=im, output_shape=(max_length, max_length), mode='constant')
        return 1 - im
    else:
        h_pad_before = int(np.floor((max_length - h)/2))
        h_pad_after = int(max_length - h - h_pad_before)
        w_pad_before = int(np.floor((max_length - w)/2))
        w_pad_after = int(max_length - w - w_pad_before)
        # add padding
        im = np.pad(im, ((h_pad_before, h_pad_after), (w_pad_before, w_pad_after)), mode='constant', constant_values=255)
        return 1 - (im / 255.)

In [0]:
#@title get images in subset and standardize

max_input_size = 128
images_all = np.zeros((temp.shape[0], max_input_size, max_input_size))
#images_all = np.zeros((N, max_input_size, max_input_size))
cwd = os.getcwd()
on_cloud = cwd.startswith(r'/home/liyunlu3')
for i in range(temp.shape[0]):
    fname = temp.iloc[i]['fname']
    file = index_image_table.loc[temp.index[i], 'image']
    if on_cloud:
        images_all[i] = standardize_image(imageio.imread('/home/liyunlu3/Documents/images/' + fname + '/' + file + '.png', pilmode='L'), max_input_size)
    else: # on colab
        images_all[i] = standardize_image(imageio.imread('/content/images/' + fname + '/' + file + '.png', pilmode='L'), max_input_size)
images_all = np.expand_dims(images_all, axis=-1)

In [0]:
#@title retrieve indices
train_index = np.load('temporary files/train_index.npy')
val_index = np.load('temporary files/val_index.npy')
test_index = np.load('temporary files/test_index.npy')

In [0]:
#@title use the indices to prepare data sets


train_data = DataSet(inputs=images_all[train_index],
                     labels=df_tsne[['x-tsne', 'y-tsne']].values[train_index],
                     aux_inputs=temp[['IOD', 'AREA']].values[train_index],
                     aux_outputs=df_tsne['region'].values[train_index])
val_data = DataSet(inputs=images_all[val_index],
                   labels=df_tsne[['x-tsne', 'y-tsne']].values[val_index],
                   aux_inputs=temp[['IOD', 'AREA']].values[val_index],
                   aux_outputs=df_tsne['region'].values[val_index])
test_data = DataSet(inputs=images_all[test_index],
                    labels=df_tsne[['x-tsne', 'y-tsne']].values[test_index],
                    aux_inputs=temp[['IOD', 'AREA']].values[test_index],
                    aux_outputs=df_tsne['region'].values[test_index])

In [0]:
cnn_layers = [8, 16, 32, 64, 128, 256, 512]
fc_layers = [128,64]

In [0]:
#@title Define CNN model with Keras

x_ = Input(shape=(max_input_size, max_input_size, 1))

x = x_
for n in cnn_layers:
    x = Conv2D(filters=n, kernel_size=3, strides=1, 
               padding='same', 
               activation='relu', name='conv'+str(n))(x)
    x = MaxPooling2D(pool_size=(2, 2), padding='valid')(x)

x = GlobalAveragePooling2D()(x)

aux_input = Input(shape=(2,), name='aux_input')
x = concatenate([x, aux_input])

i = 1
for n in fc_layers:
    x = Dense(units=n, activation='relu', name='fc'+str(i)+'_'+str(n))(x)
    i += 1

y = Dense(units=2, name='fc_final')(x)

model = Model(inputs=[x_, aux_input], outputs=y)

In [0]:
#@title Compile
model.compile(optimizer='adam', loss='mean_absolute_error')

In [0]:
model_name_prefix = 'CNN_model_v0_'

In [0]:
#@title define metrics callback
from keras.callbacks import Callback

class modelsaver(Callback):
    def on_train_begin(self, logs={}):
        self._min_mae = 6
        if len(self.model.inputs)>1:
            self._aux_in = True
        else:
            self._aux_in = False
        
    def on_epoch_end(self, epoch, logs={}):
        if l > self._min_mae:
            file = 'Keras models/CNN/'+model_name_prefix+'_%.2f.h5' % logs['val_loss']
            print('saving model at '+file)
            self.model.save(file)
            self._min_mae = l
        return

In [0]:
#@title Configure callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1, min_lr=1e-6)
saver = modelsaver（）

In [0]:
#@title train
model.fit([train_data._inputs, train_data._aux_inputs], train_data._labels, 
          epochs=300, batch_size=128, shuffle=True, 
          validation_data=([val_data._inputs, val_data._aux_inputs], val_data._labels), 
          sample_weight=df_tsne['region'].map(lambda x: 1 if x==0 else 1.5).iloc[train_index].values,
          callbacks=[reduce_lr, saver])

# Extension: Interactive fine-tuning

In [0]:
#@title define some utility functions
def get_image(guid, max_size):
    fname = xml_all.loc[guid, 'fname']
    file = index_image_table.loc[guid, 'image']
    im = standardize_image(imageio.imread('/home/liyunlu3/Documents/images/' + fname + '/' + file + '.png', 
                                          pilmode='L'), 
                           max_size)
    return (im, file)

def show_images(image1, fname1, image2, fname2):
    plt.subplots(1,2,figsize=(10,20))
    #plt.subplots(1,2)
    plt.subplot(1,2,1)
    plt.imshow(image1,cmap='gray')
    plt.title(fname1)
    plt.subplot(1,2,2)
    plt.imshow(image2,cmap='gray')
    plt.title(fname2)
    plt.show()

In [0]:
#@title Hand tuning Main process

cont = 'Y'
force = 10

while cont=='Y' or cont=='y':
    # get the first image
    # need to replace random sampling with either manual selection 
    # or local density based sampling
    if to_sample:
        # randomly choose a image with equal chance of choosing each gallery and each grade
        gallery = np.random.randint(4)
        grade = np.random.choice(['High','Low','Intermediate','Normal'],1)[0]

        guid1 = np.random.choice(df_tsne.loc[(df_tsne['GALLERY']==gallery) & (df_tsne['Grade']==grade)].index.values, 1)[0]
        im1, file1 = get_image(guid1, 128)
    # otherwise continue with the same first image
    
    im1 = np.expand_dims(im, axis=-1)
    current_output1 = model.predict(np.expand_dims(im1, axis=0), 
                                    batch_size=1)
    
    # get the second image
    # randomly sample some images, retain only the nearest to image 1
    N = 32
    sample_index = np.random.choice(df_tsne.loc[(df_tsne['GALLERY']==gallery) & (df_tsne['Grade']==grade) & (df_tsne.index!=guid1)].index.values, N, replace=False)
    
    temp_im_list = []
    temp_file_list = []
    for guid in sample_index:
        im, file = get_image(guid, 128)
        temp_im_list.append(np.expand_dims(im, axis=-1))
        temp_file_list.append(file)
    outputs = model.predict(np.stack(temp_im_list, axis=0))
    distances = np.linalg.norm(outputs - current_output1.reshape([1,2]), axis=1)
    # get the id of the nearest image
    guid2 = sample_index[np.argmin(distances)]
    im2 = temp_im_list[np.argmin(distances)]
    current_output2 = outputs[np.argmin(distances)]
    distance = distances[np.argmin(distances)]
    
    image1, file1 = get_image(guid1, 282)
    image2, file2 = get_image(guid2, 282)
    
    # plot images to compare by eye
    clear_output()
    print('Left image:', ['Tumour', 'Lymphocyte', 'Normal', 'Firoblast'][gallery], 'cell from', grade, 'grade sample')
    show_images(image1, file1, image2, file2)
    
    print('Current distance:' + "%.2f" % distance)
    
    # receive label
    signal = input('Are they similar? ')
    if signal.find('stop')>-1:
        cont='n'
        if signal.startswith('y') or signal.startswith('Y'):
            signal = 1
            to_sample = True
        else:
            signal = -1
            to_sample = False
    elif signal=='Y' or signal=='y':
        signal = 1
        to_sample = True
    else:
        signal = -1
        to_sample = False
    #records.append((guid1, guid2, signal))
    
    # set target for the training
    if signal==1:
        target1 = (current_output1 + current_output2)/2.
        target2 = target1
    else:
        target1 = current_output1 + (current_output1-current_output2) / (distance**2) * force
        target2 = current_output2 + (current_output2-current_output1) / (distance**2) * force
    
    print('output1:', current_output1, 'new target:', target1)
    print('output2:', current_output2, 'new target:', target2)
    
    # train with new targets
    model.fit(np.stack([im1, im2]), 
              np.concatenate([target1, target2], axis=0))
    
    time.sleep(2) 

In [0]:
model.save('Keras models/CNN_model_v1_tuned_.h5')